In [3]:
using Printf, CSV, DataFrames, Statistics, Glob, XLSX, Dates, Plots
# Makie

In [5]:
directory = joinpath(homedir(), "research", "GADS_Processed", "raw_data")
files = glob("gen_287_*.csv", directory)



1-element Vector{String}:
 "/home/emco4286/research/GADS_Pr" ⋯ 38 bytes ⋯ "p_1_rating_1_state_Iowa_raw.csv"

In [6]:
f = files[1]
df = DataFrame(CSV.File(f))
sort!(df, :EventStartDT)
df = df[df.EventTypeCode .!= "RS", :]
df = df[df.EventTypeCode .!= "RU", :];
mydict = Dict([("PO", 3), ("U1", 2), ("SF", 2), ("MO", 4), ("PD", 3), ("A", 1), ("F", 2), ("U2" , 2),("D4", 4),("D1", 2), ("RU", 5)])
transform!(df, :EventTypeCode => ByRow(x -> mydict[x]), :EventTypeCode);

In [7]:
start_date_g = minimum(df[!, :EventStartDT])
end_date_g = maximum(df[!, :EventEndDT])
date_range_g = [c for c in start_date_g:Hour(1):end_date_g]

end_hour_g = round(end_date_g-start_date_g, Hour).value + 1
myrange_g = [c for c in 1:1:end_hour_g]

agedict = Dict([(1, DateTime(1980, 1, 1)), (2, DateTime(1985, 1, 1)), (3, DateTime(1995, 1, 1)), (4, DateTime(2005, 1, 1)), (5, DateTime(2015, 1, 1)), (6, DateTime(2020, 1, 1))])
state = fill(1, length(myrange_g))
cause = fill(1, length(myrange_g))
age = fill(1, length(myrange_g))

birthdate = min(agedict[df[1, :CommercialDT_grp]], start_date_g)
time_vector = map(x -> round(x - start_date_g, Hour), df[!, :EventStartDT])
age_vector = map(x -> round(x - birthdate, Hour).value, date_range_g)

38688-element Vector{Int64}:
 289302
 289303
 289304
 289305
 289306
 289307
 289308
 289309
 289310
 289311
      ⋮
 327981
 327982
 327983
 327984
 327985
 327986
 327987
 327988
 327989

In [8]:
for k in [c for c in range(1, step=1, length=length(time_vector))]
    start_time_string = df[k, :EventStartDT]
    end_time_string = df[k, :EventEndDT]

    start_hour = time_vector[k].value
    duration = round(end_time_string - start_time_string, Hour).value
    end_hour = start_hour + duration

    start_index = max(1, start_hour)
    end_index = findall(x -> x == end_hour, myrange_g)[1]
    
    state[start_index:end_index] .= 0
    cause[start_index:end_index] .= df[k, :EventTypeCode_function]
end

In [9]:
u = 287
unit_type_code = df[1, :UnitTypeCode]
commercial_dt_grp = df[1, :CommercialDT_grp]
state_name = df[1, :StateName]
rating = df[1, :RatingMW_grp]

1

In [10]:
trajectory = DataFrame(x=age_vector, y=cause)

fname = @sprintf "gen_%d_type_%d_dtgrp_%d_rating_%d_state_%s_traj.csv" u unit_type_code commercial_dt_grp rating state_name
path = joinpath(homedir(), "research", "GADS_Processed", "trajectories", fname)
CSV.write(path, trajectory)

"/home/emco4286/research/GADS_Processed/trajectories/gen_287_type_100_dtgrp_1_rating_1_state_Iowa_traj.csv"

In [ ]:
f = Figure()

ax = Axis(f[1, 1], xlabel = "Date", ylabel = "Availability",
    title = "Generator #287")

colormap=Makie.Categorical(:viridis)

plot(ax, date_range_g, state,label="", xticks=false)
scatter!(date_range_g, state, zcolor=cause, markerstrokewidth=0.1, label="", xticks=false, color=1:6, colormap=colormap)

tick_years = DateTime(2013):Year(1):DateTime(2024)
DateTick = Dates.format.(tick_years, "yyyy")
plot!(xticks=(tick_years,DateTick), xtickfontsize=6)

Colorbar(fig[1, 3], limits = (0, 5),
    colormap = colormap, size = 25)

UndefVarError: UndefVarError: `Figure` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [14]:
plot(date_range_g, state,label="", xticks=false)
scatter!(date_range_g, state, zcolor=cause, markerstrokewidth=0.1, label="", xticks=false)

tick_years = DateTime(2013):Year(1):DateTime(2024)
DateTick = Dates.format.(tick_years, "yyyy")
plot!(xticks=(tick_years,DateTick), xtickfontsize=6)
savefig("gen_287_traj.png")

"/home/emco4286/research/gen_287_traj.png"